# Pronóstico adaptativo

## Descripción del problema real

Los pronósticos de los precios de la electricidad en mercados liberalizados son un insumo fundamental para la toma de decisiones dentro de las organizaciones. Fundamentalmente, los pronosticos de corto plazo son utilizados en decisiones de carácter operativo. En el caso abordado, es necesario contar con los pronósticos para definir la politica de operación de una empresa del sector eléctrico.

## Descripción del problema en términos de los datos

La carpeta `datos/precios/` contiene los precios historicos horarios de la electricidad en la Bolsa de Energía del mercado eléctrico colombiano, publicados por el operador del mercado. Se desean obtener pronósticos para el precio promedio diario para los próximos siete (7) días a la fecha actual.

## Aproximaciones posibles

En este caso, se desea evaluar la capacidad de pronóstico de un ADALINE con aprendizaje en línea.

## Requerimientos

Usted debe:

* Procesar los datos históricos para conformar el conjunto de datos. Los archivos de Excel no pueden ser modificados y actualizados directamente por el operador del sistema. Su código debe leer los archivos y crear la serie de precios promedio diarios de la electricidad.


* Determinar si el modelo debe pronosticar los precios promedios sin ninguna transformación, o si la inclusión de una transformación (logaritmo natural, raíz cúbica, raíz cuadrada, etc) resulta en un incremento de la precisión del pronóstico.


* Generar el pronóstico para los precios de los próximos siete días.


* Preparar el código para que el modelo sea entrenado usando el gradiente y el momentum.


* Determinar cuál es el número óptimo de retardos (observaciones) que el modelo debe considerar hacia atrás para producir el pronóstico.


* Determinar los valores óptimos de la tasa de aprendizaje y el momentum.


# Angel Racini Meza
## C.C 1017248189


## Importanto Librerias

In [128]:
import pandas as pd
import numpy as np 
import os
from pathlib import Path
from matplotlib import pyplot as plt
%matplotlib inline

## Preprocesando los datos

In [123]:
#Extrayendo los archivos de la carpeta precios
path  = r'C:\Users\ANGEL\Desktop\Trabajo Redes Neuronales\evaluacion-arracinim\datos\precios'
datos = os.listdir(path)

Consolidado = pd.read_excel(path+"\\"+Datos[0])
Consolidado = Consolidado.iloc[:,0:25]
Consolidado = Consolidado.dropna()
Consolidado.rename(columns=Consolidado.iloc[0], inplace = True)
Consolidado = Consolidado.drop(Consolidado.index[0])
Consolidado.reset_index()

file_number = 0

for archivo in datos:
    if file_number == 0:
        file_number = file_number + 1
        pass
    else:
        df_aux = pd.read_excel(path+"\\"+archivo)
        df_aux = df_aux.iloc[:,0:25]
        df_aux = df_aux.dropna()
        df_aux.rename(columns=df_aux.iloc[0], inplace = True)
        df_aux = df_aux.drop(Consolidado.index[0])
        Consolidado = pd.concat([Consolidado,df_aux])

In [124]:
#Obteniendo el promedio para cada dia

Precios_Promedio = Consolidado.set_index('Fecha')
Precios_Promedio = Precios_Promedio.sum(axis = 1) / 24


In [130]:
Precios_Promedio

Fecha
1995-07-21      4.924333
1995-07-22      1.269500
1995-07-23      0.953083
1995-07-24      4.305917
1995-07-25      1.149167
1995-07-26      1.108625
1995-07-27      0.585958
1995-07-28      0.499792
1995-07-29      0.927667
1995-07-30      0.669458
1995-07-31      0.679458
1995-08-01      1.106917
1995-08-02      1.077500
1995-08-03      1.517917
1995-08-04      1.786042
1995-08-05      2.383792
1995-08-06      2.387583
1995-08-07      3.707208
1995-08-08      6.650000
1995-08-09     13.991958
1995-08-10     20.041667
1995-08-11     21.191667
1995-08-12     30.710750
1995-08-13     34.327833
1995-08-14     33.190250
1995-08-15     14.513250
1995-08-16      8.664333
1995-08-17      1.813250
1995-08-18      1.941667
1995-08-19      1.101833
                 ...    
2018-04-06    106.700173
2018-04-07    114.226014
2018-04-08     97.726207
2018-04-09    103.583952
2018-04-10    107.897197
2018-04-11    133.977405
2018-04-12    110.192975
2018-04-13    116.894492
2018-04-14    111.9